# Plot positions

[direct link](#Display)

In [ ]:
import pandas as pd

In [ ]:
# parameters

combined_data_file = "combined_positions.csv"
heincke_positions = "heincke.csv"

platform_deploy_times = {
    "Littorina": "2023-04-24T05:30:00",
    "D298": "2023-04-24T08:50:00",
    "D299": "2023-04-24T08:50:00",
    "D300": "2023-04-24T08:50:00",
    "D301": "2023-04-24T09:35:00",
    "D302": "2023-04-24T09:35:00",
    "D303": "2023-04-24T09:35:00",
}

In [ ]:
import pandas as pd
import hvplot.pandas
from bokeh.models import HoverTool, SaveTool
import geoviews as gv

from functools import reduce

## Read data

In [ ]:
df = pd.read_csv(combined_data_file, parse_dates=["Timestamp", ])
df

In [ ]:
df_heincke = pd.read_csv(heincke_positions, parse_dates=["Timestamp", ])
df_heincke["Latitude_"] = df_heincke.Latitude
df_heincke["Longitude_"] = df_heincke.Longitude
df_heincke

## Plot

In [ ]:
hover = HoverTool(tooltips=[
    ("Platform", "@Platform"),
    ("Type", "@Type"),
    ("Timestamp", "@Timestamp"),
    ("Longitude", "@Longitude_"),  # display potentially redacted positions
    ("Latitude", "@Latitude_"),  # display potentially redacted positions
])

In [ ]:
savetool = SaveTool()

In [ ]:
plots = []

for npf, pf in enumerate(df.Platform.unique()):
    if npf == 0:
        _kwargs = dict(
            tiles="OSM",  
            xlim=(9.8, 12.3),
            ylim=(54.2, 55),
        )
    else:
        _kwargs = {}
    _df = df.where(
        (df.Platform == pf)
        & (df.Timestamp > platform_deploy_times[pf])
    ).dropna().sort_values("Timestamp")
    _df["Timestamp"] = _df["Timestamp"].astype(str)  # needed for proper display
    plots.append(
        (
            _df.hvplot.paths(
                x="Longitude", y="Latitude", line_width=2,
                label=pf,
                geo=True,
                **_kwargs
            )
            * _df.hvplot.points(
                x="Longitude", y="Latitude", size=25, marker="x",
                label=pf,
                hover_cols=["Platform", "Type", "Longitude_", "Latitude_", "Timestamp", ], tools=[hover, ],
                geo=True,
            )
        )
    )

In [ ]:
plots.append(
    (
        df_heincke.hvplot.paths(
            x="Longitude", y="Latitude", size=95, color="magenta", geo=True, label="Heincke", hover_cols=["Timestamp", ] 
        )
        * df_heincke.iloc[-1:].hvplot.points(
            x="Longitude", y="Latitude", size=95, color="magenta", marker=">", geo=True, label="Heincke", hover_cols=["Timestamp", ] 
        )
    )
)

In [ ]:
plot = reduce(lambda a, b: a * b, plots[1:], plots[0]).opts(frame_width=700)

## Display

In [ ]:
now = !date -Is -u
now = now[0]

In [ ]:
display(plot.opts(title=f"{now} UTC"));

## Last 3 positions

In [ ]:
pd.concat((df, df_heincke), ignore_index=True).groupby("Platform").tail(3).set_index(["Platform", "Timestamp"])[["Longitude_", "Latitude_"]]

In [ ]:
!date -Is -u